In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dem_path = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM_cleaned.tif'

resos = [20, 40, 80, 160]
catchment_no = np.arange(1, 23)
exclude = [11, 17, 18, 19]
catchs = np.setdiff1d(catchment_no, exclude)
rasters = ['channels', '5haStreams']

for raster in rasters:
    for reso in resos:
        for catch in catchs:
            stream_mask_path = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/C{catch}/{raster}.asc'
            stream_ele_path = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/C{catch}/{raster}_elev.asc'
    
            # Step 1: Read DEM and stream mask
            with rasterio.open(dem_path) as dem_src, rasterio.open(stream_mask_path) as stream_src:
                dem = dem_src.read(1)  # DEM data (0.5m resolution)
                stream_mask = stream_src.read(1)  # Stream mask data
                
                # Get metadata of the stream mask raster
                stream_meta = stream_src.meta.copy()
                stream_meta.update(crs=dem_src.crs)
                
            # Step 2: For each stream cell in stream_mask, find corresponding DEM cells
            # Stream mask has lower resolution, so we calculate corresponding DEM grid-cells
            dem_transform = dem_src.transform
            stream_transform = stream_src.transform
            
            stream_elev = np.full_like(stream_mask, -9999, dtype=np.float32)
            
            stream_cells = np.where(stream_mask == 1)  # Assuming 1 marks stream locations
            
            # Iterate over each stream cell in the lower resolution stream mask
            for i, j in zip(*stream_cells):
                # Create a 1x1 window for the current stream cell in the stream mask
                stream_window = rasterio.windows.Window(col_off=j, row_off=i, width=1, height=1)
                
                # Get the bounds of the stream cell in the stream mask's coordinate system
                stream_bounds = rasterio.windows.bounds(stream_window, stream_transform)
                
                # Convert stream bounds to corresponding DEM window using index notation
                dem_col_start, dem_row_start = ~dem_transform * (stream_bounds[0], stream_bounds[3])  # (left, top)
                dem_col_end, dem_row_end = ~dem_transform * (stream_bounds[2], stream_bounds[1])  # (right, bottom)
                    
                # Convert to integer indices for the DEM grid
                dem_row_start, dem_row_end = int(np.floor(dem_row_start)), int(np.ceil(dem_row_end))
                dem_col_start, dem_col_end = int(np.floor(dem_col_start)), int(np.ceil(dem_col_end))
                
                # Step 3: Extract DEM values within the corresponding DEM cells
                dem_subset = dem[dem_row_start:dem_row_end, dem_col_start:dem_col_end]
                        
                del dem_row_start, dem_row_end, dem_col_start, dem_col_end
            
                    
                # Flatten and remove np.nan values from DEM subset
                valid_dem_values = dem_subset[np.isfinite(dem_subset)]
                
                if len(valid_dem_values) > 0:
                    # Step 4: Calculate the first quantile (0.1)
                    #quantile_value = mstats.mquantiles(valid_dem_values, prob=0.1)[0]
                    quantile_value = np.quantile(valid_dem_values, 0.02)
                    # Assign the quantile value to the stream depth for the current stream cell
                    stream_elev[i, j] = quantile_value
                
            # Step 5: Save the new raster with the same metadata as the stream mask
            stream_meta.update(dtype=rasterio.float32, count=1)
            
            with rasterio.open(stream_ele_path, 'w', **stream_meta) as dst:
                dst.write(stream_elev, 1)

In [ ]:
resos = [20, 40, 80, 160]
catchment_no = np.arange(1, 23)
exclude = [11, 17, 18, 19]
catchs = np.setdiff1d(catchment_no, exclude)
rasters = ['channels', '5haStreams']

for raster in rasters:
    for reso in resos:
        for catch in catchs:
            stream_ele_path = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/C{catch}/{raster}_elev.asc'
            dem_path = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/C{catch}/dem_clip.tif'
            stream_depth_path = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/C{catch}/{raster}_depth.asc'
            
            # Step 1: Open the stream elevation and DEM files
            with rasterio.open(stream_ele_path) as stream_src:
                stream_ele = stream_src.read(1)
                stream_meta = stream_src.meta.copy()
            
            with rasterio.open(dem_path) as dem_src:
                dem_data = dem_src.read(1)
            
            # Step 2: Create masks
            dem_mask = (np.isfinite(dem_data)) & (dem_data != -9999)   # valid DEM
            stream_mask = (stream_ele != -9999)                       # valid stream elev
            
            # Step 3: Calculate stream depth where both valid
            stream_depth = np.where(dem_mask & stream_mask, stream_ele - dem_data, -9999)
            
            # Apply thresholds only on valid pixels
            valid = stream_depth != -9999
            stream_depth[valid & (stream_depth > 0)] = 0
            stream_depth[valid & (stream_depth < -15)] = -15
            
            # Step 4: Save output with proper nodata
            stream_meta.update(dtype=rasterio.float32, count=1, nodata=-9999)
            
            with rasterio.open(stream_depth_path, 'w', **stream_meta) as dst:
                dst.write(stream_depth.astype(rasterio.float32), 1)